<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/micro_par_bn_r_16_help_tapt_pretrain_10_epoch_lr_0004.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Version 1
*   uses par_bn (instead of seq_bn) with reduction factor of 16
*   only pretrains for 10 epochs
*   uses lr of 0.0004 because of small epoch run

In [ ]:
from time import time
start_time = time()

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters

In [ ]:
from datasets import load_dataset

dataset_name = "BigTMiami/amazon_MICRO_helpfulness_dataset_condensed"

dataset = load_dataset(dataset_name)

print(dataset_name)

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

In [ ]:
from adapters import ParBnConfig

reduction_factor = 16
adapter_config = ParBnConfig(reduction_factor=reduction_factor)
adapter_name = "micro_par_bn_v_1_pretrain"

adapter_hub_name = f"{adapter_name}_adapter"
print(adapter_hub_name)

In [ ]:
# Add a new adapter
model.add_adapter(adapter_name, config=adapter_config)

# Add head for masked language modeling
model.add_masked_lm_head(adapter_name)

# Set the adapter to be used for training
model.train_adapter(adapter_name)

print(f"Adapter Name: {adapter_name}")

In [ ]:
summary = model.adapter_summary()
print(summary)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [ ]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=adapter_hub_name + "_model",
    overwrite_output_dir=True,
    learning_rate=0.0004,
    per_device_train_batch_size=21,
    per_device_eval_batch_size=21,
    gradient_accumulation_steps=2, # Creates effective batch of 22 * 2 * 6 = 252
    num_train_epochs=10,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1, # Saves latest 2 checkpoints
    # torch_compile=True,  # Much Faster
    push_to_hub=False,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True,
    report_to="tensorboard",
)

In [ ]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=10)

In [ ]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

In [ ]:
train_results = trainer.train()
print(train_results)

In [ ]:
eval_results = trainer.evaluate(dataset["test"])
print(eval_results)

In [ ]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name,
    overwrite_adapter_card=True,
)

In [ ]:
end_time = time()
total_time = end_time - start_time
print(f"Total Time: {total_time/60:.0f} minutes OR {total_time:.0f} seconds")

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()